In [1]:
import sys
sys.path.insert(0, '../')

import uuid
import os
import time
from typing import Any, Callable, Generator

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.base import BaseOutputParser
from langchain.agents import load_tools, Tool
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

from chat.base import BaseChat, ChatHistory, Response
from chat.widget_search import ChatOutputParser, WIDGET_INSTRUCTION, SEARCH_INSTRUCTION, TEMPLATE, IDENTIFY_TEMPLATE
from chat.base import *
from index.weaviate import *
from index.widgets import *
from utils import *
from config import *
from system import *
from tools import *

set_api_key()

def send_message(resp, before_message_id=None, last_chat_message_id=None):
    if resp.operation == 'replace':
        print(resp.response)
    return 1

system = initialize(default_config)

INFO:env:Loading env file c:\Users\HARSH\Pictures\cacti-backend\tests\..\env\local.yaml


Added to class registry: chat.simple.SimpleChat
Added to class registry: chat.rephrase.RephraseChat
Added to class registry: chat.rephrase_cited.RephraseCitedChat


c:\Users\HARSH\Pictures\cacti-backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Added to class registry: chains.api_chain.IndexAPIChain
Added to class registry: chat.widget_search.WidgetSearchChat
Added to class registry: tools.index_lookup.IndexLookupTool
Added to class registry: tools.index_answer.IndexAnswerTool
Added to class registry: tools.index_widget.IndexWidgetTool
Added to class registry: index.weaviate.WeaviateIndex
Added to class registry: tools.index_app_info.IndexAppInfoTool
Added to class registry: tools.index_api_tool.IndexAPITool
Added to class registry: chat.rephrase_widget_search.RephraseWidgetSearchChat
Added to class registry: chat.rephrase_widget_search2.RephraseWidgetSearchChat
Added to class registry: chat.basic_agent.BasicAgentChat
Added to class registry: chat.fine_tuned.FineTunedChat
Added to class registry: chat.chatgpt_function_call.ChatGPTFunctionCallChat
Added to class registry: system.System
Initializing instance of type: index.weaviate.WeaviateIndex
Initializing instance of type: chat.basic_agent.BasicAgentChat
Initializing instanc

In [2]:
history = ChatHistory([], uuid.UUID, '0x123243242')
input = "Swap ETh with DAI" 
# input = "What is my balance of ETH in terms of USD"
system.chat.receive_input(history, input, send_message, uuid.UUID, uuid.UUID)

Initializing instance of type: tools.index_widget.IndexWidgetTool


> Entering new  chain...
timing - first_token:  0.76s
 The user wants to swap ETH with DAI
Prior Observations: None
Action: WidgetIndexAnswer
Action Input: Swap ETH with DAItiming - widget_index_lookup_done:  3.35s


> Entering new  chain...
Prompt after formatting:
You are a web3 widget tool. You have access to a list of widget magic commands that you can delegate work to, by invoking them and chaining them together, to provide a response to an input query. Magic commands have the structure "<|command(parameter1, parameter2, ...)|>" specifying the command and its input parameters. They can only be used with all parameters having known and assigned values, otherwise, they have to be kept secret. The command may either have a display- or a fetch- prefix. When you return a display- command, the user will see data, an interaction box, or other inline item rendered in its place. When you return a fetch- command, data is fe